In [55]:
import sys, os, itertools, sklearn
sys.path.append('/home/roquero/CausalAggregation/Code')
import pandas as pd
import numpy as np
import _pickle as pkl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model
from sklearn.linear_model import LassoCV, Lasso
from sklearn import datasets
from sklearn.metrics import auc
from scipy.interpolate import UnivariateSpline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%reload_ext autoreload
%autoreload 2
np.set_printoptions(suppress=True)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
from backfitting import Backfitting
from boosting import Boosting
from base_environment import BaseEnvironment
from collection_environment import CollectionEnvironment
from regression_method import PolynomialRegression, RandomForestRegression
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures


In [48]:
ik_h, se_h = [], (lambda input_samples:input_samples)
ik_x1, se_x1 = ['h'], (lambda input_samples: 2*input_samples[0] + input_samples[1])
ik_x2, se_x2 = ['h', 'x1'], (lambda input_samples: input_samples[0] * input_samples[1] + input_samples[2])
ik_x3, se_x3 = ['x1', 'x2'], (lambda input_samples: -input_samples[0] * 2*input_samples[1] + input_samples[2])
ik_x4, se_x4 = ['x1', 'x3'], (lambda input_samples: np.log(1+np.abs(input_samples[0])) + input_samples[1] + input_samples[2])
ik_y, se_y = ['h','x1','x2','x3','x4'], (lambda input_samples:
                                         input_samples[0] 
                                         + input_samples[1]>0
                                         - input_samples[1]*input_samples[2]*input_samples[3]
                                         + input_samples[3]*input_samples[4]
                                         + 2*input_samples[2]*input_samples[4] + input_samples[5])
ik_x5, se_x5 = ['x2', 'x4', 'y'], (lambda input_samples: 2*input_samples[0] + input_samples[1] - input_samples[2] + input_samples[3])

In [49]:
fh = {'input_keys':ik_h, 'structural_eq': se_h}
f1 = {'input_keys':ik_x1, 'structural_eq': se_x1}
f2 = {'input_keys':ik_x2, 'structural_eq': se_x2}
f3 = {'input_keys':ik_x3, 'structural_eq': se_x3}
f4 = {'input_keys':ik_x4, 'structural_eq': se_x4}
f5 = {'input_keys':ik_x5, 'structural_eq': se_x5}
fy = {'input_keys':ik_y, 'structural_eq': se_y}

sh = lambda n_samples: np.random.normal(size=n_samples)
s1 = lambda n_samples: np.random.normal(size=n_samples)
s2 = lambda n_samples: np.random.normal(size=n_samples)
s3 = lambda n_samples: np.random.normal(size=n_samples)
s4 = lambda n_samples: np.random.normal(size=n_samples)
s5 = lambda n_samples: np.random.normal(size=n_samples)
sy = lambda n_samples: np.random.normal(size=n_samples)

topo_order = ['h','x1','x2','x3','x4','y','x5']
y_key = 'y'
x_key = ['x1','x2','x3','x4','x5']

structural_equation_dict = {'x1':f1, 'x2':f2, 'x3':f3, 'y':fy, 'x4':f4, 'x5':f5, 'h':fh}
disturbance_sampler_dict = {'x1':s1, 'x2':s2, 'x3':s3, 'y':sy, 'x4':s4, 'x5':s5, 'h':sh}

base = BaseEnvironment(structural_equation_dict, disturbance_sampler_dict, topo_order, y_key, x_key)
coll_env = CollectionEnvironment(base)

In [50]:
coll_env.add_env('e1', {'x1':{'type':'independent'},'x2':{'type':'independent'}, 'x3':{'type':'independent'}}, 1000)
coll_env.add_env('e2', {'x2':{'type':'independent'},'x4':{'type':'independent'}}, 1000)
coll_env.add_env('e3', {'x3':{'type':'independent'},'x4':{'type':'independent'}}, 1000)
coll_env.add_env('e4', {'x1':{'type':'independent'},'x5':{'type':'independent'}}, 1000)
coll_env.add_env('e5', {'x4':{'type':'independent'}}, 1000)

In [ ]:
params_method = {'regression_method':'HuberRegressor',
                 'power_features': {
                     'e1':3,
                     'e2':2,
                     'e3':2,
                     'e4':2,
                     'e5':1,
                                     },
                 'include_bias':False,
                 'interactions_env': {
                     'e1':'full',
                     'e2':'int',
                     'e3':'int',
                     'e4':'int',
                     'e5':'full'
                                     }
                }

backfit = Backfitting(RandomForestRegression, max_n_iter=10, params_method = params_method)
                      
                      #{'power_features':2, 'include_bias':False})


res = backfit.fit(coll_env)

100%|██████████| 10/10 [1:22:34<00:00, 541.24s/it]

In [59]:
samples = np.random.normal(size=(6,10))
samples

array([[-1.295, -0.002, 0.182, -0.965, 0.527, 1.014, 0.431, 1.231,
        -0.876, -0.376],
       [1.060, 1.708, -1.718, 0.160, 1.488, -1.364, -0.325, -0.013,
        -0.870, 1.125],
       [0.828, 0.166, 0.683, 2.029, 1.815, 0.850, 0.455, -1.229, -0.440,
        -0.405],
       [0.032, 1.205, 1.854, -0.335, -0.137, 1.407, 0.130, 0.288, -1.171,
        0.024],
       [-1.742, 0.413, 1.048, -0.531, -0.355, 1.029, -0.703, 1.205,
        0.047, -0.042],
       [0.214, -1.436, 1.495, 0.663, -0.260, 0.309, -0.944, 0.457, 0.285,
        -0.299]])

In [60]:
backfit.true_function_y(samples)
backfit.output_function_merged(samples)

array([0.635, 4.314, -5.284, -0.168, 1.043, -2.789, 0.097, 1.173, 2.985,
       0.358])

array([0.692, 4.333, -5.293, -0.115, 1.156, -2.772, 0.126, 1.190, 3.019,
       0.378])

In [61]:
backfit.evaluate_gap()

0.001750015613496921

In [20]:
backfit.output_model_variation

{'e1': [0.33280540306460926,
  0.0,
  1.2036735031237122,
  0.15648126687706782,
  4.975558038965806,
  0.0016800261669473397,
  0.008397693166116986,
  0.0014936852701479787,
  7.52187087579448,
  7.28199820598116,
  0.00541016212974181,
  0.005099116832927655,
  7.499012581707395,
  0.0,
  0.06666455591009404,
  7.510439365922843,
  0.0009294718200405232,
  0.0009121527348833283,
  0.0009347393541834633,
  7.5589988637744545,
  0.005154538704813489,
  0.06658879404786805,
  7.576547458152493,
  0.06658213896446809,
  0.06633307530532179,
  0.005058597174107065,
  0.0009469544627888613,
  0.0,
  7.431105235257848],
 'e2': [234.56425480032144,
  0.025331755354242657,
  0.24174971410469498,
  0.13941872472456043,
  0.009577214741428391,
  0.05376693759928833,
  0.06990793328075944,
  0.0009395157384499401,
  0.002001275106221272,
  0.0011655363076074241,
  7.448231671617454,
  0.08300769408995574,
  0.08351369029505107,
  0.000959653650576012,
  7.731347112481704,
  0.001658643009355376